# Neural Network with Sklearn for Sentiment Analysis ( MLPClassifier )

In [2]:
# import library yang dibutuhkan

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import seaborn as sns
import string
import re
from collections import Counter
from tqdm import tqdm
!pip install terbilang
from terbilang import Terbilang

# Import sklearn modul

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
# from sklearn.externals import joblib

from gensim.models import Word2Vec
import pickle
import time

#import sastrawi
!pip install sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

#tokenize
!pip install nltk
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')


  Preparing metadata (setup.py) ... done
  Created wheel for terbilang: filename=terbilang-0.0.6-py3-none-any.whl size=4012 sha256=d56c2ea2ed19ed09ae9d0056ad12e5dce7a8fdabdb0ba2d0c7e684efb0f2e2b2
  Stored in directory: /root/.cache/pip/wheels/0c/d9/75/df95fb352dfd1bcb4135f3fa9aaded7a2f3ec993eb972e3253
Successfully built terbilang
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 5.3 MB/s eta 0:00:00


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
import time
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Flatten, SimpleRNN, Activation, Bidirectional, GRU
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from tensorflow.keras import backend as K

In [4]:
# Preprosses Data


list_stopwords = list(set(stopwords.words('indonesian') + stopwords.words('english') + ['ya', 'yg', 'ga', 'yuk', 'dah']))
stopwords = list_stopwords


kamus_alay = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Binar-Academy/data/new_kamusalay.csv', encoding='Latin-1',names=('alay','baku'))

# Melakukan Cleaning tweet tanpa menghapus stopwords
def text_proses(text):

    # Untuk stemmer kata
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()

    if type(text) == np.float64:
        return ""
    temp = re.sub(r'\b(USER|RT|URL)\b', '', text)
    temp = temp.lower()
    temp = re.sub(r"@[A-Za-z0-9_]+","", temp) #Menghilangkan Mention
    temp = re.sub("#[A-Za-z0-9_]+","", temp) #Menghilangkan Hashtag
    temp = re.sub(r'https://\S+', '', temp)
    # Melakukan subtitusi kata alay dengan kata baku
    temps = temp.split()
    for i in range(len(temps)):
        temp = temps[i]
        replacement = kamus_alay[kamus_alay['alay'] == temp]['baku']
        if not replacement.empty:
            temps[i] = replacement.iloc[0]
    temp = " ".join(temp.strip() for temp in temps)
    # temp = " ".join(temp.strip() for temp in temps if temp not in stopwords)

    # Mengubah angka menjadi terbaca
    # temps = temp.split()
    # for i in range(len(temps)):
    #     temp = temps[i]
    #     if temps[i].isdigit() == True :
    #         temps[i] = Terbilang().parse(temps[i]).getresult()
    #     else :
    #         temps[i]
    # temp = " ".join(temps)

    # Melakukan stemming
    tokens = word_tokenize(temp)
    temp = ' '.join([word for word in tokens ])
    temp = stemmer.stem(temp)

    temp = re.sub(r'\\x..', '', temp) #Menghilangkan Emoticon
    temp = re.sub(r'&amp;', 'dan', temp)
    temp = re.sub(r'http\S+', '', temp) #Menghilangkan link
    temp = re.sub(r"www.\S+", "", temp) #Menghilangkan link
    temp = re.sub(r"\\[n|t]", "", temp) #Menghilangkan 'enter' dan 'tab'
    temp = re.sub("[^A-Za-z\s']","", temp) #Menghilangkan yang bukan huruf
    temps = temp.split()
    temp = " ".join(temp.strip() for temp in temps)

    return temp



In [5]:
# Upload Clean_data file

train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Binar-Academy/data/data_bersih_update.csv')
display(train_df.head(1))
valid_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Binar-Academy/data/valid_preprocess.csv')
display(valid_df.head(1))
test_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Binar-Academy/data/test_preprocess.csv')
display(test_df.head(1))

,text,label,pro_text_no_angka,no_stopwords
0,warung ini dimiliki oleh pengusaha pabrik tahu...,positive,warung ini milik oleh usaha pabrik tahu yang s...,warung milik usaha pabrik puluh kenal putih ba...


,text,label,ori_text
0,meski masa kampanye sudah selesai bukan arti h...,neutral,"meski masa kampanye sudah selesai , bukan bera..."


,text,label,ori_text
0,kemarin gue datang ke tempat makan baru yang a...,negative,kemarin gue datang ke tempat makan baru yang a...


In [63]:
# print(train_df.pro_text_no_angka.duplicated().sum(), train_df.pro_text_no_angka.isna().sum())  # Data sudah clean
# print(valid_df.text.duplicated().sum(), valid_df.text.isna().sum())  # Data sudah clean
# print(test_df.text.duplicated().sum(), test_df.text.isna().sum())  # Data sudah clean


In [ ]:
# df = df.drop_duplicates(subset=['pro_text_no_angka']).reset_index(drop=True)

## membuat vector untuk inputan ke Machine Learning

### 1. Dengan menggunakan Bag of Word ( CountVectorizer )

In [8]:
# Proses Feature extraction Bag of Word ( CountVectorizer )


count_vect = CountVectorizer(ngram_range=(1,1))
count_vect2 = CountVectorizer(ngram_range=(1,2),min_df=3)

print('Feature extraction selesai')

Feature extraction selesai


### Bow Ngram (1,1)

In [12]:
train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Binar-Academy/data/data_bersih_update.csv')
valid_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Binar-Academy/data/valid_preprocess.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Binar-Academy/data/test_preprocess.csv')

# Cross Validation dengan df.pro_text_update
data_preprocessed = train_df.pro_text_no_angka.to_list() + valid_df.text.to_list()

count_vect = CountVectorizer(ngram_range=(1,1),min_df=3)

kf = KFold(n_splits=5, random_state=42, shuffle=True)

accuracies = []
y = pd.Series(train_df.label.to_list() + valid_df.label.to_list())

start_ = time.time()
for iteration, data in enumerate(kf.split(np.array(data_preprocessed)), start=1):

    data_train = np.array(data_preprocessed)[data[0]]
    data_train = count_vect.fit_transform(data_train)
    target_train = y[data[0]]

    data_test = np.array(data_preprocessed)[data[1]]
    data_test = count_vect.transform(data_test)
    target_test = y[data[1]]

    clf = MLPClassifier(early_stopping=True)
    clf.fit(data_train, target_train)
    preds = clf.predict(data_test)

    accuracy = accuracy_score(target_test, preds)

    print('Training ke-', iteration)
    print(classification_report(target_test, preds))


    accuracies.append(accuracy)

end_ = time.time()


average_accuracy = np.mean(accuracies)

# print('\n\n\n')
print('rata-rata akurasi : ', average_accuracy)
print('='*200)
print('waktu cross validation : ', end_ - start_)

X_test = test_df.text.to_list()
y_test = test_df.label

X_test_vec = count_vect.transform(X_test)

y_pred_cv = clf.predict(X_test_vec)

print(classification_report(y_test, y_pred_cv))

Training ke- 1
              precision    recall  f1-score   support

    negative       0.82      0.82      0.82       782
     neutral       0.81      0.72      0.76       242
    positive       0.90      0.91      0.91      1428

    accuracy                           0.87      2452
   macro avg       0.84      0.82      0.83      2452
weighted avg       0.86      0.87      0.86      2452

Training ke- 2
              precision    recall  f1-score   support

    negative       0.81      0.83      0.82       759
     neutral       0.83      0.75      0.79       264
    positive       0.90      0.91      0.91      1429

    accuracy                           0.87      2452
   macro avg       0.85      0.83      0.84      2452
weighted avg       0.87      0.87      0.87      2452

Training ke- 3
              precision    recall  f1-score   support

    negative       0.85      0.85      0.85       745
     neutral       0.85      0.76      0.81       250
    positive       0.92      0

### Bow ngram(1,2)

In [9]:
train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Binar-Academy/data/data_bersih_update.csv')
valid_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Binar-Academy/data/valid_preprocess.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Binar-Academy/data/test_preprocess.csv')

# Cross Validation dengan df.pro_text_update
data_preprocessed = train_df.pro_text_no_angka.to_list() + valid_df.text.to_list()

count_vect2 = CountVectorizer(ngram_range=(1,2),min_df=3)

kf = KFold(n_splits=5, random_state=42, shuffle=True)

accuracies = []
y = pd.Series(train_df.label.to_list() + valid_df.label.to_list())

start_ = time.time()
for iteration, data in enumerate(kf.split(np.array(data_preprocessed)), start=1):

    data_train = np.array(data_preprocessed)[data[0]]
    data_train = count_vect2.fit_transform(data_train)
    target_train = y[data[0]]

    data_test = np.array(data_preprocessed)[data[1]]
    data_test = count_vect2.transform(data_test)
    target_test = y[data[1]]

    clf2 = MLPClassifier(early_stopping=True)
    clf2.fit(data_train, target_train)
    preds = clf2.predict(data_test)

    accuracy = accuracy_score(target_test, preds)

    print('Training ke-', iteration)
    print(classification_report(target_test, preds))


    accuracies.append(accuracy)

end_ = time.time()


average_accuracy = np.mean(accuracies)

# print('\n\n\n')
print('rata-rata akurasi : ', average_accuracy)
print('='*100)
print('waktu cross validation : ', end_ - start_)

Training ke- 1
              precision    recall  f1-score   support

    negative       0.82      0.87      0.84       782
     neutral       0.84      0.73      0.78       242
    positive       0.93      0.92      0.92      1428

    accuracy                           0.88      2452
   macro avg       0.86      0.84      0.85      2452
weighted avg       0.88      0.88      0.88      2452

Training ke- 2
              precision    recall  f1-score   support

    negative       0.86      0.86      0.86       759
     neutral       0.80      0.78      0.79       264
    positive       0.93      0.93      0.93      1429

    accuracy                           0.89      2452
   macro avg       0.86      0.86      0.86      2452
weighted avg       0.89      0.89      0.89      2452

Training ke- 3
              precision    recall  f1-score   support

    negative       0.86      0.84      0.85       745
     neutral       0.87      0.77      0.82       250
    positive       0.92      0

In [10]:
X_test = test_df.text.to_list()
y_test = test_df.label

X_test_vec = count_vect2.transform(X_test)

y_pred_cv = clf2.predict(X_test_vec)

print(classification_report(y_test, y_pred_cv))

Waktu training - predict : 537.2742578983307
              precision    recall  f1-score   support

    negative       0.76      0.96      0.85       204
     neutral       0.77      0.47      0.58        88
    positive       0.87      0.80      0.83       208

    accuracy                           0.80       500
   macro avg       0.80      0.74      0.75       500
weighted avg       0.81      0.80      0.79       500



## BoW ngram(1,3)

In [11]:
train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Binar-Academy/data/data_bersih_update.csv')
valid_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Binar-Academy/data/valid_preprocess.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Binar-Academy/data/test_preprocess.csv')

# Cross Validation dengan df.pro_text_update
data_preprocessed = train_df.pro_text_no_angka.to_list() + valid_df.text.to_list()

count_vect3 = CountVectorizer(ngram_range=(1,3),min_df=3)

kf = KFold(n_splits=5, random_state=42, shuffle=True)

accuracies = []
y = pd.Series(train_df.label.to_list() + valid_df.label.to_list())

start_ = time.time()
for iteration, data in enumerate(kf.split(np.array(data_preprocessed)), start=1):

    data_train = np.array(data_preprocessed)[data[0]]
    data_train = count_vect3.fit_transform(data_train)
    target_train = y[data[0]]

    data_test = np.array(data_preprocessed)[data[1]]
    data_test = count_vect3.transform(data_test)
    target_test = y[data[1]]

    clf3 = MLPClassifier(early_stopping=True)
    clf3.fit(data_train, target_train)
    preds = clf3.predict(data_test)

    accuracy = accuracy_score(target_test, preds)

    print('Training ke-', iteration)
    print(classification_report(target_test, preds))


    accuracies.append(accuracy)

end_ = time.time()


average_accuracy = np.mean(accuracies)

# print('\n\n\n')
print('rata-rata akurasi : ', average_accuracy)
print('='*100)
print('waktu cross validation : ', end_ - start_)

X_test = test_df.text.to_list()
y_test = test_df.label

X_test_vec = count_vect3.transform(X_test)

y_pred_cv = clf3.predict(X_test_vec)

print(classification_report(y_test, y_pred_cv))

Training ke- 1
              precision    recall  f1-score   support

    negative       0.84      0.85      0.84       782
     neutral       0.81      0.75      0.78       242
    positive       0.92      0.92      0.92      1428

    accuracy                           0.88      2452
   macro avg       0.86      0.84      0.85      2452
weighted avg       0.88      0.88      0.88      2452

Training ke- 2
              precision    recall  f1-score   support

    negative       0.87      0.85      0.86       759
     neutral       0.81      0.76      0.78       264
    positive       0.92      0.94      0.93      1429

    accuracy                           0.89      2452
   macro avg       0.86      0.85      0.86      2452
weighted avg       0.89      0.89      0.89      2452

Training ke- 3
              precision    recall  f1-score   support

    negative       0.84      0.87      0.85       745
     neutral       0.90      0.68      0.78       250
    positive       0.92      0

## Tf-Idf ngram(1,1)

In [13]:
train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Binar-Academy/data/data_bersih_update.csv')
valid_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Binar-Academy/data/valid_preprocess.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Binar-Academy/data/test_preprocess.csv')

# Cross Validation dengan df.pro_text_update
data_preprocessed = train_df.pro_text_no_angka.to_list() + valid_df.text.to_list()

tfidf = TfidfVectorizer(ngram_range=(1,1),min_df=3)

kf = KFold(n_splits=5, random_state=42, shuffle=True)

accuracies = []
y = pd.Series(train_df.label.to_list() + valid_df.label.to_list())

start_ = time.time()
for iteration, data in enumerate(kf.split(np.array(data_preprocessed)), start=1):

    data_train = np.array(data_preprocessed)[data[0]]
    data_train = tfidf.fit_transform(data_train)
    target_train = y[data[0]]

    data_test = np.array(data_preprocessed)[data[1]]
    data_test = tfidf.transform(data_test)
    target_test = y[data[1]]

    clf_tf1 = MLPClassifier(early_stopping=True)
    clf_tf1.fit(data_train, target_train)
    preds = clf_tf1.predict(data_test)

    accuracy = accuracy_score(target_test, preds)

    print('Training ke-', iteration)
    print(classification_report(target_test, preds))


    accuracies.append(accuracy)

end_ = time.time()


average_accuracy = np.mean(accuracies)

# print('\n\n\n')
print('rata-rata akurasi : ', average_accuracy)
print('='*100)
print('waktu cross validation : ', end_ - start_)

X_test = test_df.text.to_list()
y_test = test_df.label

X_test_vec = tfidf.transform(X_test)

y_pred_cv = clf_tf1.predict(X_test_vec)

print(classification_report(y_test, y_pred_cv))

Training ke- 1
              precision    recall  f1-score   support

    negative       0.83      0.83      0.83       782
     neutral       0.82      0.69      0.75       242
    positive       0.90      0.92      0.91      1428

    accuracy                           0.87      2452
   macro avg       0.85      0.82      0.83      2452
weighted avg       0.87      0.87      0.87      2452

Training ke- 2
              precision    recall  f1-score   support

    negative       0.82      0.80      0.81       759
     neutral       0.80      0.72      0.76       264
    positive       0.89      0.92      0.91      1429

    accuracy                           0.86      2452
   macro avg       0.84      0.81      0.82      2452
weighted avg       0.86      0.86      0.86      2452

Training ke- 3
              precision    recall  f1-score   support

    negative       0.83      0.83      0.83       745
     neutral       0.88      0.71      0.79       250
    positive       0.91      0

## Tf-Idf ngram(1,2)

In [14]:
train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Binar-Academy/data/data_bersih_update.csv')
valid_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Binar-Academy/data/valid_preprocess.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Binar-Academy/data/test_preprocess.csv')

# Cross Validation dengan df.pro_text_update
data_preprocessed = train_df.pro_text_no_angka.to_list() + valid_df.text.to_list()

tfidf2 = TfidfVectorizer(ngram_range=(1,2),min_df=3)

kf = KFold(n_splits=5, random_state=42, shuffle=True)

accuracies = []
y = pd.Series(train_df.label.to_list() + valid_df.label.to_list())

start_ = time.time()
for iteration, data in enumerate(kf.split(np.array(data_preprocessed)), start=1):

    data_train = np.array(data_preprocessed)[data[0]]
    data_train = tfidf2.fit_transform(data_train)
    target_train = y[data[0]]

    data_test = np.array(data_preprocessed)[data[1]]
    data_test = tfidf2.transform(data_test)
    target_test = y[data[1]]

    clf_tf2 = MLPClassifier(early_stopping=True)
    clf_tf2.fit(data_train, target_train)
    preds = clf_tf2.predict(data_test)

    accuracy = accuracy_score(target_test, preds)

    print('Training ke-', iteration)
    print(classification_report(target_test, preds))


    accuracies.append(accuracy)

end_ = time.time()


average_accuracy = np.mean(accuracies)

# print('\n\n\n')
print('rata-rata akurasi : ', average_accuracy)
print('='*100)
print('waktu cross validation : ', end_ - start_)

X_test = test_df.text.to_list()
y_test = test_df.label

X_test_vec = tfidf2.transform(X_test)

y_pred_cv = clf_tf2.predict(X_test_vec)

print(classification_report(y_test, y_pred_cv))

Training ke- 1
              precision    recall  f1-score   support

    negative       0.83      0.84      0.83       782
     neutral       0.83      0.70      0.76       242
    positive       0.91      0.93      0.92      1428

    accuracy                           0.88      2452
   macro avg       0.85      0.82      0.84      2452
weighted avg       0.88      0.88      0.88      2452

Training ke- 2
              precision    recall  f1-score   support

    negative       0.85      0.85      0.85       759
     neutral       0.82      0.74      0.78       264
    positive       0.92      0.93      0.93      1429

    accuracy                           0.89      2452
   macro avg       0.86      0.84      0.85      2452
weighted avg       0.89      0.89      0.89      2452

Training ke- 3
              precision    recall  f1-score   support

    negative       0.86      0.84      0.85       745
     neutral       0.88      0.71      0.78       250
    positive       0.91      0

## Tf_idf ngram(1,3)

In [16]:
train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Binar-Academy/data/data_bersih_update.csv')
valid_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Binar-Academy/data/valid_preprocess.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Binar-Academy/data/test_preprocess.csv')

# Cross Validation dengan df.pro_text_update
data_preprocessed = train_df.pro_text_no_angka.to_list() + valid_df.text.to_list()

tfidf3 = TfidfVectorizer(ngram_range=(1,3),min_df=3)

kf = KFold(n_splits=5, random_state=42, shuffle=True)

accuracies = []
y = pd.Series(train_df.label.to_list() + valid_df.label.to_list())

start_ = time.time()
for iteration, data in enumerate(kf.split(np.array(data_preprocessed)), start=1):

    data_train = np.array(data_preprocessed)[data[0]]
    data_train = tfidf3.fit_transform(data_train)
    target_train = y[data[0]]

    data_test = np.array(data_preprocessed)[data[1]]
    data_test = tfidf3.transform(data_test)
    target_test = y[data[1]]

    clf_tf3 = MLPClassifier(early_stopping=True)
    clf_tf3.fit(data_train, target_train)
    preds = clf_tf3.predict(data_test)

    accuracy = accuracy_score(target_test, preds)

    print('Training ke-', iteration)
    print(classification_report(target_test, preds))


    accuracies.append(accuracy)

end_ = time.time()


average_accuracy = np.mean(accuracies)

# print('\n\n\n')
print('rata-rata akurasi : ', average_accuracy)
print('='*100)
print('waktu cross validation : ', end_ - start_)

X_test = test_df.text.to_list()
y_test = test_df.label

X_test_vec = tfidf3.transform(X_test)

y_pred_cv = clf_tf3.predict(X_test_vec)

print(classification_report(y_test, y_pred_cv))

Training ke- 1
              precision    recall  f1-score   support

    negative       0.83      0.83      0.83       782
     neutral       0.83      0.68      0.75       242
    positive       0.90      0.93      0.92      1428

    accuracy                           0.87      2452
   macro avg       0.85      0.81      0.83      2452
weighted avg       0.87      0.87      0.87      2452

Training ke- 2
              precision    recall  f1-score   support

    negative       0.84      0.84      0.84       759
     neutral       0.85      0.72      0.78       264
    positive       0.91      0.94      0.92      1429

    accuracy                           0.88      2452
   macro avg       0.87      0.83      0.85      2452
weighted avg       0.88      0.88      0.88      2452

Training ke- 3
              precision    recall  f1-score   support

    negative       0.86      0.84      0.85       745
     neutral       0.89      0.74      0.81       250
    positive       0.91      0

## Bow + Tfidf ngram(1,1)

In [17]:
train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Binar-Academy/data/data_bersih_update.csv')
valid_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Binar-Academy/data/valid_preprocess.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Binar-Academy/data/test_preprocess.csv')

# Cross Validation dengan df.pro_text_update
data_preprocessed = train_df.pro_text_no_angka.to_list() + valid_df.text.to_list()

vectorizer_1 = CountVectorizer(ngram_range=(1,1),min_df=3)
Transformer = TfidfTransformer()

kf = KFold(n_splits=5, random_state=42, shuffle=True)

accuracies = []
y = pd.Series(train_df.label.to_list() + valid_df.label.to_list())

start_ = time.time()
for iteration, data in enumerate(kf.split(np.array(data_preprocessed)), start=1):

    data_train = np.array(data_preprocessed)[data[0]]
    data_train = vectorizer_1.fit_transform(data_train)
    data_train = Transformer.fit_transform(data_train)
    target_train = y[data[0]]

    data_test = np.array(data_preprocessed)[data[1]]
    data_test = vectorizer_1.transform(data_test)
    data_test = Transformer.transform(data_test)
    target_test = y[data[1]]

    clf_tf3 = MLPClassifier(early_stopping=True)
    clf_tf3.fit(data_train, target_train)
    preds = clf_tf3.predict(data_test)

    accuracy = accuracy_score(target_test, preds)

    print('Training ke-', iteration)
    print(classification_report(target_test, preds))


    accuracies.append(accuracy)

end_ = time.time()


average_accuracy = np.mean(accuracies)

# print('\n\n\n')
print('rata-rata akurasi : ', average_accuracy)
print('='*100)
print('waktu cross validation : ', end_ - start_)

X_test = test_df.text.to_list()
y_test = test_df.label

X_test_vec = vectorizer_1.transform(X_test)
X_test_vec = Transformer.transform(X_test_vec)
y_pred_cv = clf_tf3.predict(X_test_vec)

print(classification_report(y_test, y_pred_cv))

Training ke- 1
              precision    recall  f1-score   support

    negative       0.83      0.82      0.82       782
     neutral       0.79      0.70      0.74       242
    positive       0.90      0.92      0.91      1428

    accuracy                           0.87      2452
   macro avg       0.84      0.81      0.82      2452
weighted avg       0.86      0.87      0.86      2452

Training ke- 2
              precision    recall  f1-score   support

    negative       0.80      0.82      0.81       759
     neutral       0.83      0.64      0.72       264
    positive       0.89      0.92      0.91      1429

    accuracy                           0.86      2452
   macro avg       0.84      0.79      0.81      2452
weighted avg       0.86      0.86      0.86      2452

Training ke- 3
              precision    recall  f1-score   support

    negative       0.82      0.85      0.83       745
     neutral       0.88      0.70      0.78       250
    positive       0.91      0

## Bow + Tfidf ngram(1,2)

In [18]:
train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Binar-Academy/data/data_bersih_update.csv')
valid_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Binar-Academy/data/valid_preprocess.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Binar-Academy/data/test_preprocess.csv')

# Cross Validation dengan df.pro_text_update
data_preprocessed = train_df.pro_text_no_angka.to_list() + valid_df.text.to_list()

vectorizer_2 = CountVectorizer(ngram_range=(1,2),min_df=3)
Transformer = TfidfTransformer()

kf = KFold(n_splits=5, random_state=42, shuffle=True)

accuracies = []
y = pd.Series(train_df.label.to_list() + valid_df.label.to_list())

start_ = time.time()
for iteration, data in enumerate(kf.split(np.array(data_preprocessed)), start=1):

    data_train = np.array(data_preprocessed)[data[0]]
    data_train = vectorizer_2.fit_transform(data_train)
    data_train = Transformer.fit_transform(data_train)
    target_train = y[data[0]]

    data_test = np.array(data_preprocessed)[data[1]]
    data_test = vectorizer_2.transform(data_test)
    data_test = Transformer.transform(data_test)
    target_test = y[data[1]]

    clf_tf3 = MLPClassifier(early_stopping=True)
    clf_tf3.fit(data_train, target_train)
    preds = clf_tf3.predict(data_test)

    accuracy = accuracy_score(target_test, preds)

    print('Training ke-', iteration)
    print(classification_report(target_test, preds))


    accuracies.append(accuracy)

end_ = time.time()


average_accuracy = np.mean(accuracies)

# print('\n\n\n')
print('rata-rata akurasi : ', average_accuracy)
print('='*100)
print('waktu cross validation : ', end_ - start_)

X_test = test_df.text.to_list()
y_test = test_df.label

X_test_vec = vectorizer_2.transform(X_test)
X_test_vec = Transformer.transform(X_test_vec)
y_pred_cv = clf_tf3.predict(X_test_vec)

print(classification_report(y_test, y_pred_cv))

Training ke- 1
              precision    recall  f1-score   support

    negative       0.84      0.82      0.83       782
     neutral       0.82      0.70      0.75       242
    positive       0.90      0.93      0.92      1428

    accuracy                           0.87      2452
   macro avg       0.85      0.82      0.83      2452
weighted avg       0.87      0.87      0.87      2452

Training ke- 2
              precision    recall  f1-score   support

    negative       0.84      0.85      0.84       759
     neutral       0.85      0.72      0.78       264
    positive       0.91      0.93      0.92      1429

    accuracy                           0.88      2452
   macro avg       0.87      0.83      0.85      2452
weighted avg       0.88      0.88      0.88      2452

Training ke- 3
              precision    recall  f1-score   support

    negative       0.85      0.82      0.83       745
     neutral       0.86      0.72      0.78       250
    positive       0.91      0

## Bow + Tfidf ngram(1,3)

In [19]:
train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Binar-Academy/data/data_bersih_update.csv')
valid_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Binar-Academy/data/valid_preprocess.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Binar-Academy/data/test_preprocess.csv')

# Cross Validation dengan df.pro_text_update
data_preprocessed = train_df.pro_text_no_angka.to_list() + valid_df.text.to_list()

vectorizer_3 = CountVectorizer(ngram_range=(1,3),min_df=3)
Transformer = TfidfTransformer()

kf = KFold(n_splits=5, random_state=42, shuffle=True)

accuracies = []
y = pd.Series(train_df.label.to_list() + valid_df.label.to_list())

start_ = time.time()
for iteration, data in enumerate(kf.split(np.array(data_preprocessed)), start=1):

    data_train = np.array(data_preprocessed)[data[0]]
    data_train = vectorizer_3.fit_transform(data_train)
    data_train = Transformer.fit_transform(data_train)
    target_train = y[data[0]]

    data_test = np.array(data_preprocessed)[data[1]]
    data_test = vectorizer_3.transform(data_test)
    data_test = Transformer.transform(data_test)
    target_test = y[data[1]]

    clf_tf3 = MLPClassifier(early_stopping=True)
    clf_tf3.fit(data_train, target_train)
    preds = clf_tf3.predict(data_test)

    accuracy = accuracy_score(target_test, preds)

    print('Training ke-', iteration)
    print(classification_report(target_test, preds))


    accuracies.append(accuracy)

end_ = time.time()


average_accuracy = np.mean(accuracies)

# print('\n\n\n')
print('rata-rata akurasi : ', average_accuracy)
print('='*100)
print('waktu cross validation : ', end_ - start_)

X_test = test_df.text.to_list()
y_test = test_df.label

X_test_vec = vectorizer_3.transform(X_test)
X_test_vec = Transformer.transform(X_test_vec)
y_pred_cv = clf_tf3.predict(X_test_vec)

print(classification_report(y_test, y_pred_cv))

Training ke- 1
              precision    recall  f1-score   support

    negative       0.84      0.82      0.83       782
     neutral       0.82      0.71      0.76       242
    positive       0.90      0.93      0.92      1428

    accuracy                           0.88      2452
   macro avg       0.86      0.82      0.84      2452
weighted avg       0.87      0.88      0.88      2452

Training ke- 2
              precision    recall  f1-score   support

    negative       0.85      0.84      0.84       759
     neutral       0.82      0.70      0.75       264
    positive       0.91      0.94      0.92      1429

    accuracy                           0.88      2452
   macro avg       0.86      0.82      0.84      2452
weighted avg       0.88      0.88      0.88      2452

Training ke- 3
              precision    recall  f1-score   support

    negative       0.86      0.83      0.84       745
     neutral       0.87      0.74      0.80       250
    positive       0.91      0

## LSTM + BoW

In [7]:
train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Binar-Academy/data/data_bersih_update.csv')
valid_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Binar-Academy/data/valid_preprocess.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Binar-Academy/data/test_preprocess.csv')

X_train = train_df.pro_text_no_angka.to_list()
X_valid = valid_df.text.to_list()
y_train = pd.get_dummies(train_df.label).values
y_valid = pd.get_dummies(valid_df.label).values


count_vect = CountVectorizer(ngram_range=(1,1),min_df=3)
Transformer = TfidfTransformer()

X_train_vec = count_vect.fit_transform(X_train)
X_valid_vec = count_vect.transform(X_valid)
# X_train_tfidf = Transformer.fit_transform(X_train_vec)
# X_valid_tfidf = Transformer.transform(X_valid_vec)


X_train = X_train_vec.toarray()
print(X_train.shape, y_train.shape)
X_valid = X_valid_vec.toarray()
print(X_valid.shape, y_valid.shape)

# X_train = X_train[:, :, None]
# print(X_train.shape, y_train.shape)
# X_valid = X_valid[:, :, None]
# print(X_valid.shape, y_valid.shape)

X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
print(X_train.shape, y_train.shape)
X_valid = np.reshape(X_valid, (X_valid.shape[0], X_valid.shape[1], 1))
print(X_valid.shape, y_valid.shape)

units = 128
input_shape = (X_train.shape[1],X_train.shape[2])

model = Sequential()
model.add(LSTM(units,dropout=0.2, input_shape=input_shape,))
model.add(Dense(units=64, activation='relu'))
model.add(Dense(3, activation='softmax'))

adam = optimizers.legacy.Adam(learning_rate=0.005)
model.compile(loss = 'categorical_crossentropy', optimizer=adam, metrics= ['accuracy'])
print(model.summary())
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=2, restore_best_weights=True)
history = model.fit(X_train, y_train, epochs = 10, batch_size=16, validation_data=(X_valid,y_valid), verbose=1, callbacks=[es])


(11000, 4428) (11000, 3)
(1260, 4428) (1260, 3)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 128)               66560     
                                                                 
 dense (Dense)               (None, 64)                8256      
                                                                 
 dense_1 (Dense)             (None, 3)                 195       
                                                                 
Total params: 75,011
Trainable params: 75,011
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
688/688 [==============================] - 115s 155ms/step - loss: 0.9226 - accuracy: 0.5830 - val_loss: 0.9242 - val_accuracy: 0.5833
Epoch 2/10
688/688 [==============================] - 101s 147ms/step - loss: 0.9161 - accuracy: 0.5833 - val_loss: 0

##Tensorflow LSTM + Bow + Tf-idf ngram(1,1)

In [17]:
train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Binar-Academy/data/data_bersih_update.csv')
valid_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Binar-Academy/data/valid_preprocess.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Binar-Academy/data/test_preprocess.csv')

X_train = train_df.pro_text_no_angka.to_list()
X_valid = valid_df.text.to_list()
y_train = pd.get_dummies(train_df.label).values
y_valid = pd.get_dummies(valid_df.label).values


count_vect = CountVectorizer(ngram_range=(1,1),min_df=3)
Transformer = TfidfTransformer()

X_train_vec = count_vect.fit_transform(X_train)
X_valid_vec = count_vect.transform(X_valid)
X_train_tfidf = Transformer.fit_transform(X_train_vec)
X_valid_tfidf = Transformer.transform(X_valid_vec)


X_train = X_train_tfidf.toarray()
print(X_train.shape, y_train.shape)
X_valid = X_valid_tfidf.toarray()
print(X_valid.shape, y_valid.shape)

X_train = X_train[:, :, None]
print(X_train.shape, y_train.shape)
X_valid = X_valid[:, :, None]
print(X_valid.shape, y_valid.shape)

units = 128
input_shape = (X_train.shape[1],X_train.shape[2])

model = Sequential()
model.add(LSTM(units,dropout=0.2, input_shape=input_shape))
model.add(Dense(units=64, activation='relu'))
model.add(Dense(3, activation='softmax'))

adam = optimizers.legacy.Adam(learning_rate=0.01)
model.compile(loss = 'categorical_crossentropy', optimizer=adam, metrics= ['accuracy'])
print(model.summary())
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=2, restore_best_weights=True)
history = model.fit(X_train, y_train, epochs = 10, batch_size=16, validation_data=(X_valid,y_valid), verbose=1, callbacks=[es])


(11000, 4428) (11000, 3)
(1260, 4428) (1260, 3)
(11000, 4428, 1) (11000, 3)
(1260, 4428, 1) (1260, 3)
Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_10 (LSTM)              (None, 128)               66560     
                                                                 
 dense_17 (Dense)            (None, 64)                8256      
                                                                 
 dense_18 (Dense)            (None, 3)                 195       
                                                                 
Total params: 75,011
Trainable params: 75,011
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
688/688 [==============================] - 98s 141ms/step - loss: 0.9200 - accuracy: 0.5824 - val_loss: 0.9157 - val_accuracy: 0.5833
Epoch 2/10
688/688 [==============================] - 95s 138

## LSTM + Tfidf

In [19]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
import pandas as pd

train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Binar-Academy/data/data_bersih_update.csv')
valid_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Binar-Academy/data/valid_preprocess.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Binar-Academy/data/test_preprocess.csv')

X_train = train_df.pro_text_no_angka.to_list()
X_valid = valid_df.text.to_list()
y_train = pd.get_dummies(train_df.label).values
y_valid = pd.get_dummies(valid_df.label).values

# Text feature extraction
count_vect = CountVectorizer(ngram_range=(1, 1), min_df=3)
# Transformer = TfidfTransformer()
tfidf = TfidfVectorizer()
tfidf.fit(X_train)

# X_train_vec = count_vect.fit_transform(X_train)
# X_valid_vec = count_vect.transform(X_valid)
X_train_tfidf = tfidf.transform(X_train)
X_valid_tfidf = tfidf.transform(X_valid)

# Convert to array
X_train = X_train_tfidf.toarray()
print(X_train.shape, y_train.shape)
X_valid = X_valid_tfidf.toarray()
print(X_valid.shape, y_valid.shape)

X_train = X_train[:, :, None]
print(X_train.shape, y_train.shape)
X_valid = X_valid[:, :, None]
print(X_valid.shape, y_valid.shape)

units = 128
input_shape = (X_train.shape[1],X_train.shape[2])

# Define model architecture

model = Sequential()
model.add(LSTM(units=units, dropout=0.2, input_shape=input_shape))
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=3, activation='softmax'))

# Compile and train the model
adam = optimizers.legacy.Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
print(model.summary())
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=2, restore_best_weights=True)
model.fit(X_train, y_train, epochs=10, batch_size=16, validation_data=(X_valid, y_valid), verbose=1, callbacks=[es])



(11000, 11854) (11000, 3)
(1260, 11854) (1260, 3)
(11000, 11854, 1) (11000, 3)
(1260, 11854, 1) (1260, 3)
Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_11 (LSTM)              (None, 128)               66560     
                                                                 
 dense_19 (Dense)            (None, 64)                8256      
                                                                 
 dense_20 (Dense)            (None, 3)                 195       
                                                                 
Total params: 75,011
Trainable params: 75,011
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
688/688 [==============================] - 248s 358ms/step - loss: 0.9226 - accuracy: 0.5830 - val_loss: 0.9166 - val_accuracy: 0.5833
Epoch 2/10
688/688 [==============================] - 24

## Keras SimpleRNN

In [9]:
train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Binar-Academy/data/data_bersih_update.csv')
valid_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Binar-Academy/data/valid_preprocess.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Binar-Academy/data/test_preprocess.csv')


X_train_list = train_df.pro_text_no_angka.to_list()
X_val_list = valid_df.text.to_list()
X_test_list = test_df.text.to_list()

y_train_list = train_df.label.to_list()
y_val_list = valid_df.label.to_list()
y_test_list = test_df.label.to_list()

max_features = 100_000
tokenizer = Tokenizer(num_words=max_features, split=' ', lower=True)
tokenizer.fit_on_texts(X_train_list)

X_train_token = tokenizer.texts_to_sequences(X_train_list)

vocab_size = len(tokenizer.word_index)
maxlen = max(len(x) for x in X_train_token)

# Memanggil modul pad sequances
X_train_padded = pad_sequences(X_train_token,maxlen)

Y_train_dummies = pd.get_dummies(y_train_list).values
Y_val_dummies = pd.get_dummies(y_val_list).values
Y_test_dummies = pd.get_dummies(y_test_list).values

X_train = X_train_padded
X_valid = pad_sequences(tokenizer.texts_to_sequences(X_val_list),maxlen=X_train.shape[1])
X_test = pad_sequences(tokenizer.texts_to_sequences(X_test_list),maxlen=X_train.shape[1])
y_train = Y_train_dummies
y_valid = Y_val_dummies
y_test = Y_test_dummies

embed_dim = 100
units = 128

model = Sequential()
model.add(Embedding(max_features, embed_dim, input_length= X_train.shape[1]))
model.add(SpatialDropout1D(0.7))
model.add(SimpleRNN(units, dropout=0.7))
model.add(Dense(100,activation='relu'))
model.add(Dense(3, activation='softmax'))
adam = optimizers.legacy.Adam(learning_rate=0.0001)
model.compile(loss = 'categorical_crossentropy', optimizer=adam, metrics= ['accuracy'])
print(model.summary())


model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics= ['accuracy'])

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=2, restore_best_weights=True)
history = model.fit(X_train, y_train, epochs = 20, batch_size=16, validation_data=(X_valid,y_valid), verbose=1, callbacks=[es])



Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 85, 100)           10000000  
                                                                 
 spatial_dropout1d_1 (Spatia  (None, 85, 100)          0         
 lDropout1D)                                                     
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 128)               29312     
                                                                 
 dense_3 (Dense)             (None, 100)               12900     
                                                                 
 dense_4 (Dense)             (None, 3)                 303       
                                                                 
Total params: 10,042,515
Trainable params: 10,042,515
Non-trainable params: 0
__________________________________________

KeyboardInterrupt: ignored

## LSTM keras embeding

In [13]:
train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Binar-Academy/data/data_bersih_update.csv')
valid_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Binar-Academy/data/valid_preprocess.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Binar-Academy/data/test_preprocess.csv')


X_train_list = train_df.pro_text_no_angka.to_list()
X_val_list = valid_df.text.to_list()
X_test_list = test_df.text.to_list()

y_train_list = train_df.label.to_list()
y_val_list = valid_df.label.to_list()
y_test_list = test_df.label.to_list()

max_features = 100_000
tokenizer = Tokenizer(num_words=max_features, split=' ', lower=True)
tokenizer.fit_on_texts(X_train_list)

X_train_token = tokenizer.texts_to_sequences(X_train_list)

vocab_size = len(tokenizer.word_index)
maxlen = max(len(x) for x in X_train_token)

# Memanggil modul pad sequances
X_train_padded = pad_sequences(X_train_token,maxlen)

Y_train_dummies = pd.get_dummies(y_train_list).values
Y_val_dummies = pd.get_dummies(y_val_list).values
Y_test_dummies = pd.get_dummies(y_test_list).values

X_train = X_train_padded
X_valid = pad_sequences(tokenizer.texts_to_sequences(X_val_list),maxlen=X_train.shape[1])
X_test = pad_sequences(tokenizer.texts_to_sequences(X_test_list),maxlen=X_train.shape[1])
y_train = Y_train_dummies
y_valid = Y_val_dummies
y_test = Y_test_dummies

embed_dim = 100
units = 128

model = Sequential()
model.add(Embedding(max_features, embed_dim, input_length= X_train.shape[1]))
model.add(SpatialDropout1D(0.7))
model.add(LSTM(units, dropout=0.7))
model.add(Dense(100))
model.add(Dense(3, activation='softmax'))
adam = optimizers.legacy.Adam(learning_rate=0.0001)
model.compile(loss = 'categorical_crossentropy', optimizer=adam, metrics= ['accuracy'])
print(model.summary())


model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics= ['accuracy'])

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=2, restore_best_weights=True)
history = model.fit(X_train, y_train, epochs = 20, batch_size=16, validation_data=(X_valid,y_valid), verbose=1, callbacks=[es])



Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 85, 100)           10000000  
                                                                 
 spatial_dropout1d_3 (Spatia  (None, 85, 100)          0         
 lDropout1D)                                                     
                                                                 
 lstm_2 (LSTM)               (None, 128)               117248    
                                                                 
 dense_7 (Dense)             (None, 100)               12900     
                                                                 
 dense_8 (Dense)             (None, 3)                 303       
                                                                 
Total params: 10,130,451
Trainable params: 10,130,451
Non-trainable params: 0
__________________________________________

In [14]:
predictions = model.predict(X_test)
y_pred = predictions
matrix_test = classification_report(y_test.argmax(axis=1), y_pred.argmax(axis=1))
print('Testing selesai !')
print(matrix_test)

16/16 [==============================] - 0s 4ms/step
Testing selesai !
              precision    recall  f1-score   support

           0       0.75      0.96      0.84       204
           1       0.87      0.51      0.64        88
           2       0.87      0.79      0.83       208

    accuracy                           0.81       500
   macro avg       0.83      0.75      0.77       500
weighted avg       0.82      0.81      0.80       500



### Bidirectional LSTM

In [10]:
train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Binar-Academy/data/data_bersih_update.csv')
valid_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Binar-Academy/data/valid_preprocess.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Binar-Academy/data/test_preprocess.csv')


X_train_list = train_df.pro_text_no_angka.to_list()
X_val_list = valid_df.text.to_list()
X_test_list = test_df.text.to_list()

y_train_list = train_df.label.to_list()
y_val_list = valid_df.label.to_list()
y_test_list = test_df.label.to_list()

max_features = 100_000
tokenizer = Tokenizer(num_words=max_features, split=' ', lower=True)
tokenizer.fit_on_texts(X_train_list)

X_train_token = tokenizer.texts_to_sequences(X_train_list)

vocab_size = len(tokenizer.word_index)
maxlen = max(len(x) for x in X_train_token)

# Memanggil modul pad sequances
X_train_padded = pad_sequences(X_train_token,maxlen)

Y_train_dummies = pd.get_dummies(y_train_list).values
Y_val_dummies = pd.get_dummies(y_val_list).values
Y_test_dummies = pd.get_dummies(y_test_list).values

X_train = X_train_padded
X_valid = pad_sequences(tokenizer.texts_to_sequences(X_val_list),maxlen=X_train.shape[1])
X_test = pad_sequences(tokenizer.texts_to_sequences(X_test_list),maxlen=X_train.shape[1])
y_train = Y_train_dummies
y_valid = Y_val_dummies
y_test = Y_test_dummies

embed_dim = 100
units = 128

model = Sequential()
model.add(Embedding(max_features, embed_dim, input_length= X_train.shape[1]))
model.add(SpatialDropout1D(0.7))
model.add(Bidirectional(LSTM(units, dropout=0.7)))
model.add(Dense(100))
model.add(Dense(3, activation='softmax'))
adam = optimizers.legacy.Adam(learning_rate=0.0001)
model.compile(loss = 'categorical_crossentropy', optimizer=adam, metrics= ['accuracy'])
print(model.summary())


model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics= ['accuracy'])

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=2, restore_best_weights=True)
history = model.fit(X_train, y_train, epochs = 20, batch_size=16, validation_data=(X_valid,y_valid), verbose=1, callbacks=[es])



Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 85, 100)           10000000  
                                                                 
 spatial_dropout1d_2 (Spatia  (None, 85, 100)          0         
 lDropout1D)                                                     
                                                                 
 bidirectional (Bidirectiona  (None, 256)              234496    
 l)                                                              
                                                                 
 dense_5 (Dense)             (None, 100)               25700     
                                                                 
 dense_6 (Dense)             (None, 3)                 303       
                                                                 
Total params: 10,260,499
Trainable params: 10,260,499


In [12]:
predictions = model.predict(X_test)
y_pred = predictions
matrix_test = classification_report(y_test.argmax(axis=1), y_pred.argmax(axis=1))
print('Testing selesai !')
print(matrix_test)

16/16 [==============================] - 0s 7ms/step
Testing selesai !
              precision    recall  f1-score   support

           0       0.82      0.94      0.87       204
           1       0.82      0.51      0.63        88
           2       0.85      0.87      0.86       208

    accuracy                           0.83       500
   macro avg       0.83      0.77      0.79       500
weighted avg       0.83      0.83      0.82       500



### GRU

In [15]:
train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Binar-Academy/data/data_bersih_update.csv')
valid_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Binar-Academy/data/valid_preprocess.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Binar-Academy/data/test_preprocess.csv')


X_train_list = train_df.pro_text_no_angka.to_list()
X_val_list = valid_df.text.to_list()
X_test_list = test_df.text.to_list()

y_train_list = train_df.label.to_list()
y_val_list = valid_df.label.to_list()
y_test_list = test_df.label.to_list()

max_features = 100_000
tokenizer = Tokenizer(num_words=max_features, split=' ', lower=True)
tokenizer.fit_on_texts(X_train_list)

X_train_token = tokenizer.texts_to_sequences(X_train_list)

vocab_size = len(tokenizer.word_index)
maxlen = max(len(x) for x in X_train_token)

# Memanggil modul pad sequances
X_train_padded = pad_sequences(X_train_token,maxlen)

Y_train_dummies = pd.get_dummies(y_train_list).values
Y_val_dummies = pd.get_dummies(y_val_list).values
Y_test_dummies = pd.get_dummies(y_test_list).values

X_train = X_train_padded
X_valid = pad_sequences(tokenizer.texts_to_sequences(X_val_list),maxlen=X_train.shape[1])
X_test = pad_sequences(tokenizer.texts_to_sequences(X_test_list),maxlen=X_train.shape[1])
y_train = Y_train_dummies
y_valid = Y_val_dummies
y_test = Y_test_dummies

embed_dim = 100
units = 128

model = Sequential()
model.add(Embedding(max_features, embed_dim, input_length= X_train.shape[1]))
model.add(SpatialDropout1D(0.7))
model.add(GRU(units, dropout=0.7))
model.add(Dense(100))
model.add(Dense(3, activation='softmax'))
adam = optimizers.legacy.Adam(learning_rate=0.0001)
model.compile(loss = 'categorical_crossentropy', optimizer=adam, metrics= ['accuracy'])
print(model.summary())


model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics= ['accuracy'])

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=2, restore_best_weights=True)
history = model.fit(X_train, y_train, epochs = 20, batch_size=16, validation_data=(X_valid,y_valid), verbose=1, callbacks=[es])

print()
predictions = model.predict(X_test)
y_pred = predictions
matrix_test = classification_report(y_test.argmax(axis=1), y_pred.argmax(axis=1))
print('Testing selesai !')
print(matrix_test)


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 85, 100)           10000000  
                                                                 
 spatial_dropout1d_4 (Spatia  (None, 85, 100)          0         
 lDropout1D)                                                     
                                                                 
 gru (GRU)                   (None, 128)               88320     
                                                                 
 dense_9 (Dense)             (None, 100)               12900     
                                                                 
 dense_10 (Dense)            (None, 3)                 303       
                                                                 
Total params: 10,101,523
Trainable params: 10,101,523
Non-trainable params: 0
__________________________________________

### GRU Bidirectional

In [16]:
train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Binar-Academy/data/data_bersih_update.csv')
valid_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Binar-Academy/data/valid_preprocess.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Binar-Academy/data/test_preprocess.csv')


X_train_list = train_df.pro_text_no_angka.to_list()
X_val_list = valid_df.text.to_list()
X_test_list = test_df.text.to_list()

y_train_list = train_df.label.to_list()
y_val_list = valid_df.label.to_list()
y_test_list = test_df.label.to_list()

max_features = 100_000
tokenizer = Tokenizer(num_words=max_features, split=' ', lower=True)
tokenizer.fit_on_texts(X_train_list)

X_train_token = tokenizer.texts_to_sequences(X_train_list)

vocab_size = len(tokenizer.word_index)
maxlen = max(len(x) for x in X_train_token)

# Memanggil modul pad sequances
X_train_padded = pad_sequences(X_train_token,maxlen)

Y_train_dummies = pd.get_dummies(y_train_list).values
Y_val_dummies = pd.get_dummies(y_val_list).values
Y_test_dummies = pd.get_dummies(y_test_list).values

X_train = X_train_padded
X_valid = pad_sequences(tokenizer.texts_to_sequences(X_val_list),maxlen=X_train.shape[1])
X_test = pad_sequences(tokenizer.texts_to_sequences(X_test_list),maxlen=X_train.shape[1])
y_train = Y_train_dummies
y_valid = Y_val_dummies
y_test = Y_test_dummies

embed_dim = 100
units = 128

model = Sequential()
model.add(Embedding(max_features, embed_dim, input_length= X_train.shape[1]))
model.add(SpatialDropout1D(0.7))
model.add(Bidirectional(GRU(units, dropout=0.7)))
model.add(Dense(100))
model.add(Dense(3, activation='softmax'))
adam = optimizers.legacy.Adam(learning_rate=0.0001)
model.compile(loss = 'categorical_crossentropy', optimizer=adam, metrics= ['accuracy'])
print(model.summary())


model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics= ['accuracy'])

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=2, restore_best_weights=True)
history = model.fit(X_train, y_train, epochs = 20, batch_size=16, validation_data=(X_valid,y_valid), verbose=1, callbacks=[es])

print()
predictions = model.predict(X_test)
y_pred = predictions
matrix_test = classification_report(y_test.argmax(axis=1), y_pred.argmax(axis=1))
print('Testing selesai !')
print(matrix_test)


Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 85, 100)           10000000  
                                                                 
 spatial_dropout1d_5 (Spatia  (None, 85, 100)          0         
 lDropout1D)                                                     
                                                                 
 bidirectional_1 (Bidirectio  (None, 256)              176640    
 nal)                                                            
                                                                 
 dense_11 (Dense)            (None, 100)               25700     
                                                                 
 dense_12 (Dense)            (None, 3)                 303       
                                                                 
Total params: 10,202,643
Trainable params: 10,202,643


In [17]:
train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Binar-Academy/data/data_bersih_update.csv')
valid_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Binar-Academy/data/valid_preprocess.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Binar-Academy/data/test_preprocess.csv')


X_train_list = train_df.pro_text_no_angka.to_list()
X_val_list = valid_df.text.to_list()
X_test_list = test_df.text.to_list()

y_train_list = train_df.label.to_list()
y_val_list = valid_df.label.to_list()
y_test_list = test_df.label.to_list()

max_features = 100_000
tokenizer = Tokenizer(num_words=max_features, split=' ', lower=True)
tokenizer.fit_on_texts(X_train_list)

X_train_token = tokenizer.texts_to_sequences(X_train_list)

vocab_size = len(tokenizer.word_index)
maxlen = max(len(x) for x in X_train_token)

# Memanggil modul pad sequances
X_train_padded = pad_sequences(X_train_token,maxlen)

Y_train_dummies = pd.get_dummies(y_train_list).values
Y_val_dummies = pd.get_dummies(y_val_list).values
Y_test_dummies = pd.get_dummies(y_test_list).values

X_train = X_train_padded
print(X_train.shape)
X_valid = pad_sequences(tokenizer.texts_to_sequences(X_val_list),maxlen=X_train.shape[1])
X_test = pad_sequences(tokenizer.texts_to_sequences(X_test_list),maxlen=X_train.shape[1])
y_train = Y_train_dummies
y_valid = Y_val_dummies
y_test = Y_test_dummies

# embed_dim = 100
# units = 128

# model = Sequential()
# model.add(Embedding(max_features, embed_dim, input_length= X_train.shape[1]))
# model.add(SpatialDropout1D(0.7))
# model.add(Bidirectional(GRU(units, dropout=0.7)))
# model.add(Dense(100))
# model.add(Dense(3, activation='softmax'))
# adam = optimizers.legacy.Adam(learning_rate=0.0001)
# model.compile(loss = 'categorical_crossentropy', optimizer=adam, metrics= ['accuracy'])
# print(model.summary())


# model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics= ['accuracy'])

# es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=2, restore_best_weights=True)
# history = model.fit(X_train, y_train, epochs = 20, batch_size=16, validation_data=(X_valid,y_valid), verbose=1, callbacks=[es])

# print()
# predictions = model.predict(X_test)
# y_pred = predictions
# matrix_test = classification_report(y_test.argmax(axis=1), y_pred.argmax(axis=1))
# print('Testing selesai !')
# print(matrix_test)


(11000, 85)


## Dense keras embedding

In [9]:
train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Binar-Academy/data/data_bersih_update.csv')
valid_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Binar-Academy/data/valid_preprocess.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Binar-Academy/data/test_preprocess.csv')


X_train_list = train_df.pro_text_no_angka.to_list()
X_val_list = valid_df.text.to_list()
X_test_list = test_df.text.to_list()

y_train_list = train_df.label.to_list()
y_val_list = valid_df.label.to_list()
y_test_list = test_df.label.to_list()

max_features = 100_000
tokenizer = Tokenizer(num_words=max_features, split=' ', lower=True)
tokenizer.fit_on_texts(X_train_list)

X_train_token = tokenizer.texts_to_sequences(X_train_list)

vocab_size = len(tokenizer.word_index)
maxlen = max(len(x) for x in X_train_token)

# Memanggil modul pad sequances
X_train_padded = pad_sequences(X_train_token,maxlen)

Y_train_dummies = pd.get_dummies(y_train_list).values
Y_val_dummies = pd.get_dummies(y_val_list).values
Y_test_dummies = pd.get_dummies(y_test_list).values

X_train = X_train_padded
X_valid = pad_sequences(tokenizer.texts_to_sequences(X_val_list),maxlen=X_train.shape[1])
X_test = pad_sequences(tokenizer.texts_to_sequences(X_test_list),maxlen=X_train.shape[1])
y_train = Y_train_dummies
y_valid = Y_val_dummies
y_test = Y_test_dummies

embed_dim = 100
units = 128

model = Sequential()
model.add(Embedding(max_features, embed_dim, input_length= X_train.shape[1]))
model.add(SpatialDropout1D(0.7))
# model.add(LSTM(units, dropout=0.7))
model.add(Flatten())
model.add(Dense(100))
model.add(Dense(3, activation='softmax'))
adam = optimizers.legacy.Adam(learning_rate=0.0001)
model.compile(loss = 'categorical_crossentropy', optimizer=adam, metrics= ['accuracy'])
print(model.summary())


model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics= ['accuracy'])

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=2, restore_best_weights=True)
history = model.fit(X_train, y_train, epochs = 20, batch_size=16, validation_data=(X_valid,y_valid), verbose=1, callbacks=[es])



Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 85, 100)           10000000  
                                                                 
 spatial_dropout1d_3 (Spatia  (None, 85, 100)          0         
 lDropout1D)                                                     
                                                                 
 flatten (Flatten)           (None, 8500)              0         
                                                                 
 dense_6 (Dense)             (None, 100)               850100    
                                                                 
 dense_7 (Dense)             (None, 3)                 303       
                                                                 
Total params: 10,850,403
Trainable params: 10,850,403
Non-trainable params: 0
__________________________________________

In [9]:
train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Binar-Academy/data/data_bersih_update.csv')
valid_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Binar-Academy/data/valid_preprocess.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Binar-Academy/data/test_preprocess.csv')

X_train_list = train_df.pro_text_no_angka.to_list()
X_val_list = valid_df.text.to_list()
X_test_list = test_df.text.to_list()

y_train_list = train_df.label.to_list()
y_val_list = valid_df.label.to_list()
y_test_list = test_df.label.to_list()

y_train = train_df.label
y_valid = valid_df.label
y_test = test_df.label

max_features = 100_000
tokenizer = Tokenizer(num_words=max_features, split=' ', lower=True)
tokenizer.fit_on_texts(X_train_list)

X_train_token = tokenizer.texts_to_sequences(X_train_list)

vocab_size = len(tokenizer.word_index)
maxlen = max(len(x) for x in X_train_token)

# Memanggil modul pad sequances
X_train_padded = pad_sequences(X_train_token,maxlen)

Y_train_dummies = pd.get_dummies(y_train_list).values
Y_val_dummies = pd.get_dummies(y_val_list).values
Y_test_dummies = pd.get_dummies(y_test_list).values

X_train = X_train_padded
X_valid = pad_sequences(tokenizer.texts_to_sequences(X_val_list),maxlen=X_train.shape[1])
X_test = pad_sequences(tokenizer.texts_to_sequences(X_test_list),maxlen=X_train.shape[1])
# y_train = Y_train_dummies
# y_valid = Y_val_dummies
# y_test = Y_test_dummies

embed_dim = 100
units = 128
embbed = Embedding(input_dim = max_features, output_dim = embed_dim, input_length= X_train.shape[1])
X_train_embed = embbed(X_train)
X_train_embed = X_train_embed.reshape(X_train_embed.shape[0],-1)
X_valid_embed = embbed(X_valid)
X_valid_embed = X_valid_embed.reshape(X_valid_embed.shape[0],-1)

clf_embed = MLPClassifier(hidden_layer_sizes=(256,128),early_stopping=True)
clf_embed.fit(X_train_embed, y_train)
preds = clf_embed.predict(X_valid_embed)

accuracy = accuracy_score(y_valid, preds)
print(accuracy)
print(classification_report(y_valid, preds))


    # clf_tf3 = MLPClassifier(early_stopping=True)
    # clf_tf3.fit(data_train, target_train)
    # preds = clf_tf3.predict(data_test)

    # accuracy = accuracy_score(target_test, preds)

    # print('Training ke-', iteration)
    # print(classification_report(target_test, preds))


# model = Sequential()
# model.add(Embedding(max_features, embed_dim, input_length= X_train.shape[1]))
# model.add(SpatialDropout1D(0.7))
# # model.add(LSTM(units, dropout=0.7))
# model.add(Flatten())
# model.add(Dense(100))
# model.add(Dense(3, activation='softmax'))
# adam = optimizers.legacy.Adam(learning_rate=0.0001)
# model.compile(loss = 'categorical_crossentropy', optimizer=adam, metrics= ['accuracy'])
# print(model.summary())

# model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics= ['accuracy'])

# es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=2, restore_best_weights=True)
# history = model.fit(X_train, y_train, epochs = 20, batch_size=16, validation_data=(X_valid,y_valid), verbose=1, callbacks=[es])



0.7309523809523809
              precision    recall  f1-score   support

    negative       0.60      0.69      0.64       394
     neutral       0.48      0.31      0.38       131
    positive       0.84      0.83      0.84       735

    accuracy                           0.73      1260
   macro avg       0.64      0.61      0.62      1260
weighted avg       0.73      0.73      0.73      1260



In [7]:
from tensorflow.python.ops.numpy_ops import np_config

np_config.enable_numpy_behavior()

In [ ]:
with open('model_BoW.pickle', 'wb') as f:
    pickle.dump(clf, f)

with open('bow_tokenizer.pickle', 'wb') as f:
    pickle.dump(count_vect, f)

In [55]:
# real world test
original_text = '''


daftar harga dispenser 2018 terbaik yang ada di blibli.com memuat banyak produk . salah satu nya adalah miyako wdp - 200 dispenser . produk ini merupakan dispenser dengan penempatan galon di bawah sehingga anda tidak perlu mengangkat galon saat akan mengisi air .


                '''
input_ = text_proses(original_text)
text = count_vect.transform([input_])
result = clf.predict(text)[0]

print('Sentence :', input_)
print('Sentiment :',result)


Sentence : daftar harga dispenser baik yang ada di blibli com muat banyak produk salah satu nya adalah miyako wdp dispenser produk ini rupa dispenser dengan tempat galon di bawah sehingga anda tidak perlu angkat galon saat akan isi air
Sentiment : positive


### 2. Dengan menggunakan Tf-IDF ( TfidfVectorizer )

In [58]:
# Proses Feature extraction Tf-IDF ( TfidfVectorizer )
tfidf2 = TfidfVectorizer(ngram_range=(1,1))
# X_tfidf = tfidf.fit_transform(data_preprocessed)
print('Feature Extraction Selesai')

Feature Extraction Selesai


In [59]:
# Cross Validation dengan df.pro_text
data_preprocessed = train_df.pro_text_no_angka.to_list() + valid_df.text.to_list()

kf = KFold(n_splits=5, random_state=42, shuffle=True)

accuracies = []
y = pd.Series(train_df.label.to_list() + valid_df.label.to_list())

start_ = time.time()
for iteration, data in (enumerate(kf.split(np.array(data_preprocessed)), start=1)):

    data_train = np.array(data_preprocessed)[data[0]]
    data_train = tfidf2.fit_transform(data_train)
    target_train = y[data[0]]

    data_test = np.array(data_preprocessed)[data[1]]
    data_test = tfidf2.transform(data_test)
    target_test = y[data[1]]

    clf3 = MLPClassifier(random_state=None, verbose=False, early_stopping=True)
    clf3.fit(data_train, target_train)
    preds = clf3.predict(data_test)

    accuracy = accuracy_score(target_test, preds)

    print('Training ke-', iteration)
    print(classification_report(target_test, preds))


    accuracies.append(accuracy)

end_ = time.time()


average_accuracy = np.mean(accuracies)

# print('\n\n\n')
print('rata-rata akurasi : ', average_accuracy)
print('='*100)
print('Lama waktu cross validation :', round(end_ - start_), 'detik')

Training ke- 1
              precision    recall  f1-score   support

    negative       0.83      0.81      0.82       782
     neutral       0.83      0.69      0.75       242
    positive       0.89      0.93      0.91      1428

    accuracy                           0.87      2452
   macro avg       0.85      0.81      0.83      2452
weighted avg       0.86      0.87      0.86      2452

Training ke- 2
              precision    recall  f1-score   support

    negative       0.81      0.82      0.81       759
     neutral       0.84      0.70      0.76       264
    positive       0.90      0.92      0.91      1429

    accuracy                           0.86      2452
   macro avg       0.85      0.81      0.83      2452
weighted avg       0.86      0.86      0.86      2452

Training ke- 3
              precision    recall  f1-score   support

    negative       0.84      0.83      0.84       745
     neutral       0.91      0.73      0.81       250
    positive       0.90      0

In [60]:
X_test = test_df.text.to_list()
y_test = test_df.label

X_test_vec = tfidf2.transform(X_test)

y_pred_cv = clf3.predict(X_test_vec)
print(classification_report(y_test, y_pred_cv))

              precision    recall  f1-score   support

    negative       0.69      0.88      0.77       204
     neutral       0.84      0.35      0.50        88
    positive       0.76      0.74      0.75       208

    accuracy                           0.73       500
   macro avg       0.76      0.66      0.67       500
weighted avg       0.75      0.73      0.72       500



In [ ]:
with open('model_tfidf.pickle', 'wb') as f:
    pickle.dump(clf, f)

with open('tfidf_tokenizer.pickle', 'wb') as f:
    pickle.dump(tfidf, f)

In [62]:
# real world test
original_text = '''

daftar harga dispenser 2018 terbaik yang ada di blibli.com memuat banyak produk . salah satu nya adalah miyako wdp - 200 dispenser . produk ini merupakan dispenser dengan penempatan galon di bawah sehingga anda tidak perlu mengangkat galon saat akan mengisi air .


                '''
input_ = text_proses(original_text)
text = tfidf2.transform([input_])
result = clf3.predict(text)[0]

print('Sentence :', input_)
print('Sentiment :',result)


Sentence : daftar harga dispenser baik yang ada di blibli com muat banyak produk salah satu nya adalah miyako wdp dispenser produk ini rupa dispenser dengan tempat galon di bawah sehingga anda tidak perlu angkat galon saat akan isi air
Sentiment : positive


### 3. Dengan menggunakan gensim model ( word2vec )

In [ ]:
# Preprocess the text (tokenization)
nltk.download('punkt')
data_preprocessed = df.pro_text_no_angka.to_list()
corpus = [nltk.word_tokenize(doc) for doc in data_preprocessed]

# Train Word2Vec model
model = Word2Vec(corpus, vector_size=400, window=5, min_count=1, workers=4)

# Generate document embeddings
X_w2v = [model.wv[doc].mean(axis=0) for doc in corpus]

print('Feature Extraction Selesai')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Feature Extraction Selesai


In [ ]:
# Cross Validation

kf = KFold(n_splits=5, random_state=42, shuffle=True)

accuracies = []
y = df.label

start_ = time.time()
for iteration, data in (enumerate(kf.split(X_w2v), start=1)):

    data_train = np.array(X_w2v)[data[0]]
    target_train = np.array(y)[data[0]]

    data_test = np.array(X_w2v)[data[1]]
    target_test = np.array(y)[data[1]]

    clf = MLPClassifier(hidden_layer_sizes=(200,200) ,random_state=None, verbose=False, early_stopping=True)
    clf.fit(data_train, target_train)
    preds = clf.predict(data_test)

    accuracy = accuracy_score(target_test, preds)

    print('Training ke-', iteration)
    print(classification_report(target_test, preds))

    accuracies.append(accuracy)

end_ = time.time()
average_accuracy = np.mean(accuracies)

print('rata-rata akurasi : ', average_accuracy)
print('='*100)

print('Lama waktu cross validation :', round(end_ - start_), 'detik')

Training ke- 1
              precision    recall  f1-score   support

    negative       0.74      0.79      0.76       655
     neutral       0.75      0.66      0.70       216
    positive       0.90      0.89      0.90      1311

    accuracy                           0.84      2182
   macro avg       0.80      0.78      0.79      2182
weighted avg       0.84      0.84      0.84      2182

Training ke- 2
              precision    recall  f1-score   support

    negative       0.79      0.77      0.78       671
     neutral       0.74      0.73      0.73       240
    positive       0.89      0.91      0.90      1271

    accuracy                           0.85      2182
   macro avg       0.81      0.80      0.80      2182
weighted avg       0.84      0.85      0.84      2182

Training ke- 3
              precision    recall  f1-score   support

    negative       0.77      0.75      0.76       696
     neutral       0.76      0.56      0.64       213
    positive       0.88      0

In [ ]:
# test dengan real world case

original_text = '''

	bagi2 sembako gratis tim @basuki_btp itu melanggar Pasal 73 Ayat 1 UU No.10 Thn 2016 #pilkadadki2017


                '''
input_ = text_proses(original_text)
text = [nltk.word_tokenize(input_)]
model_w2v = Word2Vec(text, vector_size=400, window=5, min_count=1, workers=4)
data_w2v = [model_w2v.wv[doc].mean(axis=0) for doc in text]
result = clf.predict(data_w2v)[0]

print('Sentence : ',input_)
print('Sentiment : ',result)


Sentence :  bagi bagi sembako gratis tim itu langgar pasal  ayat  undang undang no  tahun
Sentiment :  negative


## 4. Kombinasi Bag of word & Tfidf

In [21]:
# X_train, X_test, y_train, y_test = train_test_split(df.pro_text_no_angka, df.label, test_size=0.25, random_state=46, stratify=df.label)
X_train = train_df.pro_text_no_angka
X_valid = valid_df.text
X_test = test_df.text
y_train = train_df.label
y_valid = valid_df.label
y_test = test_df.label



In [22]:
Vectorize = CountVectorizer(ngram_range=(1,1))
X_train_vec = Vectorize.fit_transform(X_train)
X_valid_vec = Vectorize.transform(X_valid)
X_test_vec = Vectorize.transform(X_test)

X_train_vec

<11000x11854 sparse matrix of type '<class 'numpy.int64'>'
	with 266267 stored elements in Compressed Sparse Row format>

In [ ]:
# with open ('Vectorize_kombinasi.pickle','wb') as f:
#   pickle.dump(Vectorize,f)

In [ ]:
# with open ('Vectorize_kombinasi.pickle', 'rb') as f:
#   vectorx = pickle.load(f)

In [24]:
Transformer = TfidfTransformer()

X_train_tfidf = Transformer.fit_transform(X_train_vec)
X_valid_tfidf = Transformer.transform(X_valid_vec)
X_test_tfidf = Transformer.transform(X_test_vec)

X_train_tfidf

<11000x11854 sparse matrix of type '<class 'numpy.float64'>'
	with 266267 stored elements in Compressed Sparse Row format>

In [ ]:
# with open ('Transformer_kombinasi.pickle','wb') as f:
#   pickle.dump(Transformer,f)

In [27]:

import time

start_ = time.time()
clf = MLPClassifier(hidden_layer_sizes=(128,128,128),early_stopping=True)
clf.fit(X_train_tfidf, y_train)

y_pred = clf.predict(X_valid_tfidf)
end_ = time.time()
print('Waktu training - predict :', end_ - start_)
print(classification_report(y_valid, y_pred))

Waktu training - predict : 43.291741609573364
              precision    recall  f1-score   support

    negative       0.81      0.86      0.83       394
     neutral       0.87      0.72      0.79       131
    positive       0.91      0.91      0.91       735

    accuracy                           0.87      1260
   macro avg       0.86      0.83      0.84      1260
weighted avg       0.88      0.87      0.87      1260



In [28]:
y_pred2 = clf.predict(X_test_tfidf)
end_ = time.time()
print('Waktu training - predict :', end_ - start_)
print(classification_report(y_test, y_pred2))

Waktu training - predict : 57.93999934196472
              precision    recall  f1-score   support

    negative       0.73      0.87      0.80       204
     neutral       0.72      0.44      0.55        88
    positive       0.78      0.76      0.77       208

    accuracy                           0.75       500
   macro avg       0.75      0.69      0.71       500
weighted avg       0.75      0.75      0.74       500



In [32]:
len(train_df.pro_text_no_angka.to_list() + valid_df.text.to_list())

12260

In [44]:
y = pd.Series(train_df.label.to_list() + valid_df.label.to_list())
type(y)

pandas.core.series.Series

In [41]:
type(train_df.label.to_list() + valid_df.label.to_list())

list

In [45]:
# Cross Validation dengan df.pro_text
data_preprocessed = train_df.pro_text_no_angka.to_list() + valid_df.text.to_list()

kf = KFold(n_splits=5, random_state=42, shuffle=True)

accuracies = []
y = pd.Series(train_df.label.to_list() + valid_df.label.to_list())

start_ = time.time()
for iteration, data in enumerate(kf.split(np.array(data_preprocessed)), start=1):

    data_train = np.array(data_preprocessed)[data[0]]
    data_train_vec = Vectorize.fit_transform(data_train)
    data_train_tfidf = Transformer.fit_transform(data_train_vec)
    target_train = y[data[0]]

    data_test = np.array(data_preprocessed)[data[1]]
    data_test_vec = Vectorize.transform(data_test)
    data_test_tfidf = Transformer.transform(data_test_vec)
    target_test = y[data[1]]

    clf = MLPClassifier(hidden_layer_sizes=(128,128,128),verbose=False, early_stopping=True)
    clf.fit(data_train_tfidf, target_train)
    preds = clf.predict(data_test_tfidf)

    accuracy = accuracy_score(target_test, preds)

    print('Training ke-', iteration)
    print(classification_report(target_test, preds))

    accuracies.append(accuracy)
end_ = time.time()
average_accuracy = np.mean(accuracies)

print('rata-rata akurasi : ', average_accuracy)
print('='*100)
print('Lama waktu cross validation :', round(end_ - start_), 'detik')


Training ke- 1
              precision    recall  f1-score   support

    negative       0.81      0.84      0.83       782
     neutral       0.85      0.69      0.76       242
    positive       0.90      0.91      0.90      1428

    accuracy                           0.87      2452
   macro avg       0.85      0.81      0.83      2452
weighted avg       0.87      0.87      0.86      2452

Training ke- 2
              precision    recall  f1-score   support

    negative       0.80      0.83      0.81       759
     neutral       0.81      0.70      0.75       264
    positive       0.90      0.90      0.90      1429

    accuracy                           0.86      2452
   macro avg       0.83      0.81      0.82      2452
weighted avg       0.86      0.86      0.86      2452

Training ke- 3
              precision    recall  f1-score   support

    negative       0.82      0.86      0.84       745
     neutral       0.87      0.70      0.78       250
    positive       0.91      0

In [47]:
X_test_vec = Vectorize.transform(X_test)
X_test_tfidf = Transformer.transform(X_test_vec)

y_pred_cv = clf.predict(X_test_tfidf)
end_ = time.time()
print('Waktu training - predict :', end_ - start_)
print(classification_report(y_test, y_pred_cv))

Waktu training - predict : 433.981228351593
              precision    recall  f1-score   support

    negative       0.70      0.89      0.79       204
     neutral       0.76      0.35      0.48        88
    positive       0.78      0.75      0.77       208

    accuracy                           0.74       500
   macro avg       0.75      0.66      0.68       500
weighted avg       0.74      0.74      0.72       500



In [ ]:
# with open('model_mlp_kombinasi.pickle', 'wb') as f:
#     pickle.dump(clf, f)

with open ('Vectorize_kombinasi.pickle','wb') as f:
  pickle.dump(Vectorize,f)

with open ('Transformer_kombinasi.pickle','wb') as f:
  pickle.dump(Transformer,f)


In [49]:
# real world test
original_text = '''

daftar harga dispenser 2018 terbaik yang ada di blibli.com memuat banyak produk . salah satu nya adalah miyako wdp - 200 dispenser . produk ini merupakan dispenser dengan penempatan galon di bawah sehingga anda tidak perlu mengangkat galon saat akan mengisi air .

                '''
input_ = text_proses(original_text)
text = Vectorize.transform([input_])
text = Transformer.transform(text)
result = clf.predict(text)[0]

print('Sentence : ',input_)
print('Sentiment : ',result)


Sentence :  daftar harga dispenser baik yang ada di blibli com muat banyak produk salah satu nya adalah miyako wdp dispenser produk ini rupa dispenser dengan tempat galon di bawah sehingga anda tidak perlu angkat galon saat akan isi air
Sentiment :  positive


In [50]:
# dir(Transformer)
Transformer.get_feature_names_out().shape

(11162,)

In [51]:
Vectorize.get_feature_names_out().shape

(11162,)